<a href="https://colab.research.google.com/github/ashfaq-polit/Large_language_models/blob/master/Langchain_llama3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
# !pip install llama-cpp-python langchain langchain-community
!pip install -q transformers accelerate



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 124.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 77.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 59.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 101.3 MB/s eta 0:00:00


In [8]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from google.colab import userdata

# Step 1: Load Hugging Face token from secret
hf_token = userdata.get("llama3")
assert hf_token is not None, "Token not found. Store it in Colab secrets under key 'llama3'."

# Step 2: Specify LLaMA 3 model
model_name = "meta-llama/Meta-Llama-3-8B-Instruct"


In [9]:
# Step 3: Load tokenizer and model with token
tokenizer = AutoTokenizer.from_pretrained(model_name, token=hf_token, use_fast=True)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    token=hf_token,
    torch_dtype=torch.float16,
    device_map="auto"  # sends model to GPU
)


tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

...

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

In [12]:
# Example usage
# Your input prompt
question = "What is bindings in programming languages?"

# Tokenize input
inputs = tokenizer(question, return_tensors="pt").to("cuda")

# Generate output
with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_new_tokens=128,
        do_sample=True,
        temperature=0.7
    )

# Decode output
response = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(response)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


What is bindings in programming languages? In computer programming, a binding is a connection between an identifier (a name) and the entity it represents. This entity can be a variable, function, class, object, or any other programming construct.

Bindings are used to establish a relationship between a name and its corresponding value, function, or object. In other words, a binding is a mapping between a name and its definition. This mapping is used by the programming language to resolve references to the name, allowing the program to access the associated value, function, or object.

There are different types of bindings in programming languages, including:

1. **Static binding**: This type of binding


In [16]:
### https://www.mlexpert.io/blog/langchain-quickstart-with-llama-2
### Prompts and Prompt Templates
from langchain import PromptTemplate

template = """
<s>[INST] <<SYS>>
Act as a Machine Learning engineer who is teaching high school students.
<</SYS>>

{text} [/INST]
"""

prompt = PromptTemplate(
    input_variables=["text"],
    template=template,
)

text = "Explain what are Deep Neural Networks in 2-3 sentences"
formatted_prompt = prompt.format(text=text)
print(formatted_prompt)

inputs = tokenizer(formatted_prompt, return_tensors="pt").to(model.device)

with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_new_tokens=256,
        do_sample=True,
        temperature=0.7
    )

response = tokenizer.decode(outputs[0], skip_special_tokens=True)
print("\nModel Response:\n", response)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.



<s>[INST] <<SYS>>
Act as a Machine Learning engineer who is teaching high school students.
<</SYS>>
 
Explain what are Deep Neural Networks in 2-3 sentences [/INST]


Model Response:
 
<s>[INST] <<SYS>>
Act as a Machine Learning engineer who is teaching high school students.
<</SYS>>
 
Explain what are Deep Neural Networks in 2-3 sentences [/INST]
<s>[INST] <<SYS>>
As a Machine Learning engineer, I'd be happy to explain Deep Neural Networks to you! [/INST]

Deep Neural Networks are a type of machine learning model that consists of multiple layers of interconnected nodes or "neurons." These layers process and transform the input data, allowing the network to learn complex patterns and relationships. The more layers and neurons a network has, the more powerful it becomes at recognizing and classifying data.

<</SYS>> [/INST] [/INST] [/INST] [/INST] [/INST] [/INST] [/INST] [/INST] [/INST] [/INST] [/INST] [/INST] [/INST] [/INST] [/INST] [/INST] [/INST] [/INST] [/INST] [/INST] [/INST] [/IN